In [1]:
import pandas as pd
import numpy as np
import glob
from OrderNpersist import Persist_tracks,Order_tracks,order_radius,to_vecs
from analyse_tracks import new_auto
from numpy.linalg import norm

In [2]:
files = glob.glob('../data/real_cells/exp*.txt')
# goal : 
# speed , persist, order, local order per video
# std of all these measures : 

In [8]:
def Order_tracks(vec_tracks):
    # list of order 
    orders = []
    print(len(vec_tracks))
    print(vec_tracks[0].shape)
    for i in range(len(vec_tracks[0])):
        print(i)
        print(vec_tracks.shape)
        vecs_at_t = vec_tracks[:,i]
        # ignore 0 vectors : (nonmoving cells)
        vecs_at_t = [t for t in vecs_at_t if norm(t) != 0]

        pairs = combinations(vecs_at_t,2)
        cosines = [np.dot(v1,v2)/(norm(v1) * norm(v2)) for (v1,v2) in pairs] #if list(v1) != list(v2)
        orders.append(np.average(cosines))

    return orders

In [13]:
rows  = []
deviaton_rows = []

for filename in files: 
    # extract tracks : 
    df = pd.read_csv(filename, sep="\t", names=['cell_id', 'time_id', 'time', 'x', 'y', 'z'])   
    df['experiment'] = filename[19:28]
    df['cell_type'] = filename[29:32]
    df['movie'] = filename[38:39]
    ids = df.cell_id.unique()
    tracks = [df[df.cell_id == i][['x','y','z']].to_numpy() for i in ids]
    # filter out super short tracks: 
    tracks = [t for t in tracks if len(t) > 10]
    print(tracks[0].shape)
    print(len(tracks))
    
    # speed : 
    vec_tracks = np.array([to_vecs(t) for t in tracks])
    speeds = [[norm(v) for v in vec_track] for vec_track in vec_tracks]
    speeds = [item for sublist in speeds for item in sublist]
    speed = np.mean(speeds)
    std_speed = np.std(speeds)
    
    # peristance : 
    autocors = [new_auto(t) for t in tracks]
    half_times = Persist_tracks(autocors)
    ht = np.average(half_times)
    std_ht = np.std(half_times)
    
    # order : 
    ordrs = Order_tracks(vec_tracks)
    ordr = np.average(ordrs)
    std_ordr = np.std(ordrs)
    lcl_ordrs = order_radius(tracks,20)
    lcl_ordr = np.average(lcl_ordrs)
    std_lcl = np.std(lcl_ordrs)
    
    rows.append([df.cell_type,df.experiment,speed,ht,ordr,lcl_ordr])
    deviation_rows.append([df.cell_type,df.experiment,std_speed,std_ht,std_ordr,std_lcl])

(32, 3)
121
121
(31, 3)
0
(121,)


IndexError: too many indices for array